In [24]:
import pandas as pd 

In [25]:
file_path ='itmo_sber.csv'
data = pd.read_csv('itmo_sber.csv')

In [26]:
print(data.isnull().sum())

Unnamed: 0                    0
Задача                        0
Обстановка                    0
Оптимальный план              0
Задача en                     0
Обстановка en                 0
Оптимальный план en           0
Предсказанный план            0
Успех предсказанного плана    0
dtype: int64


In [27]:
print(data.head())

   Unnamed: 0                          Задача  \
0           0   Приготовить горячий шоколад.    
1           1     Заказать доставку еды домой   
2           2  Реставрация старого комода\r\n   
3           3            Приготовить сырники.   
4           4               Нарезать колбасу.   

                                          Обстановка  \
0  кухня, молоко, какао, сахар, кастрюля, ложка, ...   
1  мобильный телефон, ресторанное меню, интернет,...   
2  Старый комод, повреждения, шпатель, шкурка, ма...   
3  Кухня, столешница, творог, мука, яйцо, сахар, ...   
4  Кухня, колбаса, нож, доска для резки, холодиль...   

                                    Оптимальный план  \
0  Откройте дверь. Зайдите на кухню. Закрйоте две...   
1  Найдите стол. Подойдите к столу. Найдите мобил...   
2  Подойдите к комоду.Найдите шкурку.Возьмите шку...   
3  Подойти к столешнице. Найти творог. Взять твор...   
4  Подойти к холодильнику. Открыть холодильник. Н...   

                               

In [28]:
print(data['Успех предсказанного плана'].value_counts())

Успех предсказанного плана
1    3234
0    1600
Name: count, dtype: int64


In [29]:
print(data.describe())

        Unnamed: 0  Успех предсказанного плана
count  4834.000000                 4834.000000
mean   2416.500000                    0.669011
std    1395.599931                    0.470618
min       0.000000                    0.000000
25%    1208.250000                    0.000000
50%    2416.500000                    1.000000
75%    3624.750000                    1.000000
max    4833.000000                    1.000000


In [30]:
from sklearn.feature_extraction.text import CountVectorizer

In [31]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Задача'])

In [32]:
print(X.shape)

(4834, 5402)


In [33]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words and word.isalpha()]
    return ' '.join(words)

data['Задача'] = data['Задача'].apply(preprocess_text)
data['Обстановка'] = data['Обстановка'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
# Инициализация векторизатора
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Объединяем текстовые данные для векторизации
combined_text = data['Задача'] + " " + data['Обстановка']
X = tfidf_vectorizer.fit_transform(combined_text)

# Целевая переменная
y = data['Успех предсказанного плана']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the model: {accuracy}')

Accuracy of the model: 0.6783867631851086


In [37]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# Объединение текстовых полей
data['combined_text'] = data['Задача en'] + ' ' + data['Обстановка en'] + ' ' + data['Предсказанный план']

# Функция для очистки и нормализации текста
def clean_text(text):
    text = text.lower()  # Приведение к нижнему регистру
    text = re.sub(r'\s+', ' ', text)  # Удаление лишних пробелов
    text = re.sub(r'[^\w\s]', '', text)  # Удаление специальных символов
    return text

# Применение функции очистки к каждой строке объединенного текстового поля
data['cleaned_text'] = data['combined_text'].apply(clean_text)

# Подготовка данных и целевой переменной
X = data['cleaned_text']
y = data['Успех предсказанного плана']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Векторизация текста с использованием TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Ограничение на количество признаков для упрощения модели
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

X_train_tfidf.shape, X_test_tfidf.shape


((3867, 1000), (967, 1000))

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Обучение модели логистической регрессии
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_tfidf, y_train)

# Предсказание на тестовой выборке
y_pred = log_reg.predict(X_test_tfidf)

# Расчет точности модели
accuracy = accuracy_score(y_test, y_pred)
accuracy


0.6763185108583247

In [39]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


def objective(trial):
    # Определите пространство поиска гиперпараметров
    C = trial.suggest_float("C", 0.0001, 10000)
    penalty = trial.suggest_categorical("penalty", ["l2", "none"])
    max_iter = trial.suggest_int("max_iter", 100, 1000)

    # Модель логистической регрессии
    model = LogisticRegression(
        C=C,
        penalty=penalty,
        max_iter=max_iter
    )

    # Обучение
    model.fit(X_train_tfidf, y_train)

    # Предсказание
    y_pred = model.predict(X_test_tfidf)

    f1 = f1_score(y_test, y_pred)

    return f1


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

#лучшие гиперпараметры
best_params = study.best_params
best_C = best_params["C"]
best_penalty = best_params["penalty"]
best_max_iter = best_params["max_iter"]

#модель с лучшими гиперпараметрами
best_model = LogisticRegression(
    C=best_C,
    penalty=best_penalty,
    max_iter=best_max_iter
)

best_model.fit(X, y)

y_pred = best_model.predict(X_test)
#Лучшие гиперпараметры для логистической регрессии
print(best_params)

# Оценим качество
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Вычисление F1-меры
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

# Вычисление ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc:.2f}")

[I 2024-04-04 16:11:45,240] A new study created in memory with name: no-name-77366017-8d89-4b7d-bc2a-6cd8ae987f1a
[I 2024-04-04 16:11:46,364] Trial 0 finished with value: 0.7218045112781954 and parameters: {'C': 7352.7693075660445, 'penalty': 'l2', 'max_iter': 924}. Best is trial 0 with value: 0.7218045112781954.
C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[I 2024-04-04 16:11:46,806] Trial 1 finished with value: 0.7218045112781954 and parameters: {'C': 4650.51694376259, 'penalty': 'l2', 'max_ite

ValueError: could not convert string to float: 'prepare hot chocolate kitchen milk cocoa sugar pan spoon cabinet mug chair sink the mug is in the closet the sink is located next to the stove milk cocoa sugar and a spoon are on the table the pan is on the stove walk to the closet open the closet find a mug grab a mug close the closet come to the table find a spoon grab a spoon find milk grab milk pour milk into a mug find cocoa grab cocoa add cocoa to the mug find sugar grab sugar add sugar to the mug place the cocoa on the table place sugar on the table grab a spoon stir the contents of the mug with a spoon place the spoon on the table find a saucepan grab a saucepan pour the contents of the mug into the pan place the mug on the table walk to the stove place the pan on the stove switch on the stove heat the milk prepare hot chocolate switch off the stove'